In [1]:
# from sklearn.datasets import load_boston /////deprecated\
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
load_boston = fetch_openml(data_id=531, as_frame=True, parser='pandas')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [2]:
# Gather Data

boston_dataset = load_boston
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)
log_prices = np.log(boston_dataset.target)

target = log_prices.to_frame(name=['PRICES'])
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33


In [3]:

property_stats = np.ndarray(shape=(1,11))

PTRATIO_IDX = 8
RM_IDX = 4
CHAS_IDX = 3

property_stats = features.astype(float).mean().values.reshape(1,11)
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [4]:
regr = LinearRegression().fit(features.astype(float), target)
fitted_vals = regr.predict(features.astype(float))

#Challange: calculate the MSE and RMSE using sklearn

MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)
RMSE

0.18751213519713034

In [5]:
def get_log_estimate(nr_rooms, 
                     students_per_classroom,
                     next_to_river=False,
                     high_confidence=True) : 
    #Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom

    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    

    #Make prediction
    log_estimate = regr.predict(property_stats)[0][0]

    #Calc Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68


    return log_estimate, upper_bound, lower_bound, interval

In [6]:
get_log_estimate(3,18, next_to_river=True, high_confidence=False)

/Users/Albert/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


(2.432338507682168, 2.6198506428792983, 2.2448263724850377, 68)

In [7]:
np.median(boston_dataset.target)

21.2

In [19]:
#Challange: write the python code that converts the log price estimate using 1970s prices
#as well as the upper and the lower bounds to today's prices. Round the value to the nearest
#1000 dollars

ZILLOW_MEDIAN_PRICE = 584.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, uppper, lower, conf = get_log_estimate(9, students_per_classroom=15, next_to_river=False, high_confidence=False)
#Convert to today's dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**uppper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR
dollar_est

#Round the dollar values to nearest 1000
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)
rounded_est

print(f'The estimated property value is {rounded_est}.')
print(f'At {conf}% confidence the valuation range is')
print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')


/Users/Albert/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


1245000.0

In [28]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):

    """
    Estimate the price of a property in Boston.\n

    Keywords:\n
        rm -- Number of rooms;\n
        ptratio -- Number of pupils per teacher in the classroom for the school in the area; \n
        chas -- True if the property is next to the river;\n
        large_range -- True for a 95% prediction interval, False for a 68% prediction interval\n

    """

    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again!')
        return
    
    ZILLOW_MEDIAN_PRICE = 584.3
    SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

    log_est, uppper, lower, conf = get_log_estimate(rm, students_per_classroom = ptratio, next_to_river = chas, high_confidence = large_range)
    
    #Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**uppper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR
    dollar_est

    #Round the dollar values to nearest 1000
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)
    rounded_est

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [25]:
get_dollar_estimate(rm=2, ptratio=15, chas=True)

The estimated property value is 321000.0.
At 95% confidence the valuation range is
USD 220000.0 at the lower end to USD 467000.0 at the high end.


/Users/Albert/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [29]:
import boston_valuation as val

In [30]:
val.get_dollar_estimate(6, 12, True)

The estimated property value is 516000.0.
At 95% confidence the valuation range is
USD 354000.0 at the lower end to USD 750000.0 at the high end.


/Users/Albert/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
